# Validación de modelos

Al entrenar y evaluar modelos de ML, uno de los aspectos más importantes es **validar** correctamente para estimar de forma fiable cómo se comportará el modelo con datos nuevos (no vistos). 

Aunque la forma más básica es la separación de datos en *train* y *test* (train_test_split), existen diferentes técnicas de validación más sofisticadas que se utilizan para maximizar el uso de los datos disponibles y obtener una estimación más estable del desempeño del modelo.

La **validación cruzada** en Scikit-learn es una técnica que se utiliza para evaluar el desempeño de un modelo predictivo, dividiendo los datos en conjuntos de entrenamiento y prueba múltiples veces de manera sistemática. Esto permite obtener una estimación más confiable y robusta del rendimiento del modelo que simplemente dividir los datos una sola vez.

**Métodos para realizar validación de modelos**

1. train_test_split
2. cross_val_score
3. cross_validate
4. cross_val_predict
5. make_scorer

**Clases para realizar distintos tipos de validación cruzada**

1. KFold
2. StratifiedKFold
3. LeaveOneOut
4. RepeatedStratifiedKFold
5. GroupKFold
6. TimeSeriesSplit

In [1]:
from sklearn.model_selection import cross_val_score, train_test_split

cross_val_score()

TypeError: missing a required argument: 'estimator'